# Generate CREs associated for ENGRAM

In [ ]:
def gen_synCRE(sequence, motif, repeat, spacing, dist):
    """
    Generate a synthetic enhancer sequence by repeating a motif with specified spacing and distance from the minimal promoter.

    Args:
        sequence (str): The base DNA sequence.
        motif (str): The motif to repeat.
        repeat (int): Number of motif repeats.
        spacing (int): Number of bases between motifs.
        dist (int): Distance from the end of the last motif to the minimal promoter.

    Returns:
        str: The constructed synthetic enhancer sequence.
    """
    motif_len = len(motif)
    total_motif_len = motif_len * repeat
    total_spacing_len = spacing * (repeat - 1)
    enhancer_len = total_motif_len + total_spacing_len

    start = len(sequence) - enhancer_len - dist
    parts = [sequence[:start]]
    # Add motif and spacing in order
    for idx in range(repeat):
        parts.append(motif)
        if idx < repeat - 1:
            # Calculate the start and end for the spacing region
            spacing_start = start + (motif_len + spacing) * idx + motif_len
            spacing_end = spacing_start + spacing
            parts.append(sequence[spacing_start:spacing_end])

    # Add the minimal promoter region if dist > 0
    if dist == 0:
        return ''.join(parts)
    else:
        return ''.join(parts) + sequence[-dist:]
def reverse_complement(seq):
    """Return the reverse complement of a DNA sequence."""
    complement = str.maketrans('ACGTNacgtn', 'TGCANtgcan')
    return seq.translate(complement)[::-1]


In [16]:
# Sequence components for ENGRAM cloning.
BsaI_fwd, BsaI_rev = 'atactacGGTCTCagaac' , 'actgcGAGACCgtaatgc'
BsmBI_filler = 'CACTAgagacgattaaATGGACAGCAtgcgtctcTGTCC'
DTT_peg = 'CACCATCATCCNNNNNCGTGCTCACCATC' # HA + typewriter_key + symbol (5N) + PBS. longer than 5N might reduce the efficiency
HEK3_peg = 'TCTGCCATCANNNNNCGTGCTCAGTCTG' # HA + symbol （5N) + PBS. 5N can be replaced with any length of N's

In [17]:
# example CRE sequence (Wnt signaling)
CRE = 'AGATCAAAGGGTTTAAGATCAAAGGGCTTAAGATCAAAGGGTATAAGATCAAAGGGCCTAAGATCAAAGGGACTAAGATCAAAGGGTTTAAGATCAAAGGGCTTAAGATCAAAGGGCCTA'
oligo = BsaI_fwd + CRE + BsmBI_filler + HEK3_peg + BsaI_rev

In [ ]:
# Example to generage synthetic CREs with different motifs and repeats
motifs = ["gccgcagtggccgcagtg"] # 2x repeat
seq = 'GCAAACAGTCCCCATGTTCACATTAGGTTCCCAATCAGTATTTCCCATCAAGGGAGAAGCTTAGGCTGGGGAATCCTGCATACATAATTCATACCCATTATCTCAGGCTTGCTTTCTTCAACTTAAGTAGATAGGTGTATATAAAGAGACACAGCATGAAGAAGAAAGATAATAAAGATGGGCCCTGGAGAGTGACCTCTGGTGACCACAAACCTGACCTC' # mix of seq3505 seq9474, can be shortened
oligos = []
for s in motifs:
    for i in range(1,8):
        synCRE = gen_synCRE(seq,s,i,15,0)
        oligo = BsaI_fwd + synCRE + BsmBI_filler + HEK3_peg + BsaI_rev
        oligos.append([f'ZF-ErBb_2x{i}', oligo]) 

# Retriving symbols from the DNA Tape. 

This section provides information about how to process recording data from fastq files.\
Briefly, barcodes were extracted and counted from raw fastq files by pattern matching function regex. \
There are two types of tapes: synHEK3-Tape and DNA Typewriter 5x-Tape. 

## Processing synHEK3 Tape

In [ ]:
import subprocess
import os
path = '../' # data dir here
samples =[s for s in os.listdir(path) if 'fastq.gz' in s if 'R1' in s] # for HEK3-TAPE

for s in samples:
    cmd = (
        f"zcat {path}{s} | awk '{{if(NR%4==2) print $0}}' | "
        f"awk ' match($1,/CATCA([ATCG]{{0,6}})CGTGC/) {{print substr($1, RSTART+5,RLENGTH-10)}}' | "
        f"sort -k1n | uniq -c | sort -k1nr | "
        f"awk '{{a[NR]=$2;x+=(b[NR]=$1)}}END{{while(++i<=NR) print a[i]\"\\t\"b[i]\"\\t\"100*b[i]/x}}' "
        f"> {path}{s.split('_S')[0]}_bc_count.tsv&"
    )
    p=subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
    p.wait()

## Processing DNA Typewriter 
DTT data was first aligned to the reference and then analyzed by the custom python script to extract and count the barcode

In [ ]:
path = '../' # or pulse 
samples =[s for s in os.listdir(path) if '.gz' in s and 'R1' in s]
# don't forget to bwa index DTT.fasta
ref = '../analysis/DTT.fasta' # DTT.fasta is the reference genome
for s in samples:
    cmd = (
        f"bwa mem {ref} {path}{s} "
        f"| samtools view -F 0x904 | awk '$3 == \"5X_TAPE\" {{print $1\"\\t\"$3\"\\t\"$10; next}}' "
        f"> {path}{s.split('_S')[0]}.txt&"
    )
    p=subprocess.Popen(cmd,stdout=subprocess.DEVNULL,stderr=subprocess.STDOUT, shell=True)
    p.wait()

In [ ]:
path = '../'
samples =sorted([s.split('.txt')[0] for s in os.listdir(path) if '.txt' in s and 'dox' not in s])

pipeline = ''
for sample in samples:
    cmd = f"python ../analysis/PE_analysis_5XTAPE_wnt_dox.py {sample} {path} &"
    print (cmd)